In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from text_model.text_embedding import compute_sklearn_features

In [2]:
#for i in range(30):
    #index = np.random.randint(df_all.shape[0])
    #print(df_all['search_query'][index])
    #print(df_all['text'][index])
    #print('\n')

In [3]:
X_sklearn, y_sklearn = compute_sklearn_features()

Finished loading word embedding weights.
Finished loading dataframes.
Computing sklearn features:
Finished


In [4]:
X_sklearn.shape

(295424, 50)

In [5]:
import gensim
gensim.__version__

'2.1.0'

In [10]:
_RANDOM_SEED = 0
X_sklearn_train, X_sklearn_valid, y_sklearn_train, y_sklearn_valid = train_test_split(X_sklearn, y_sklearn, 
                                                                                      test_size=0.2,
                                                                                      random_state=_RANDOM_SEED)

In [11]:
logreg = LogisticRegression()
logreg.fit(X_sklearn_train, y_sklearn_train)
print(accuracy_score(logreg.predict(X_sklearn_train), y_sklearn_train))
print(accuracy_score(logreg.predict(X_sklearn_valid), y_sklearn_valid))

0.519279509518
0.517864094102


In [14]:
forest = RandomForestClassifier(max_depth=12)
forest.fit(X_sklearn_train, y_sklearn_train)
print(accuracy_score(forest.predict(X_sklearn_train), y_sklearn_train))
print(accuracy_score(forest.predict(X_sklearn_valid), y_sklearn_valid))

0.582438784966
0.511043412034


In [ ]:
from text_model.text_embedding import main
main()

Finished loading word embedding weights.
Finished loading dataframes.
Training:


In [ ]:
Epoch: 10
0%  loss = 1.150, accuracy = 0.555, speed = 430 pps
10%  loss = 1.061, accuracy = 0.593, speed = 5735 pps
20%  loss = 1.063, accuracy = 0.590, speed = 5972 pps
30%  loss = 1.065, accuracy = 0.589, speed = 6057 pps
40%  loss = 1.064, accuracy = 0.589, speed = 6093 pps
50%  loss = 1.064, accuracy = 0.589, speed = 6116 pps
60%  loss = 1.063, accuracy = 0.589, speed = 6122 pps
70%  loss = 1.063, accuracy = 0.589, speed = 6135 pps
80%  loss = 1.062, accuracy = 0.589, speed = 6145 pps
90%  loss = 1.063, accuracy = 0.589, speed = 6145 pps
100%  loss = 1.063, accuracy = 0.589, speed = 6150 pps

In [15]:
from datasets.download_images import download_im_with_text

search_query = 'surprised'
start = 0
end = 100
download_im_with_text(search_query, start, end, dataset_dir='data', subdir='photos')

Finished loading word embedding weights.


In [16]:
from datasets.convert_images_tfrecords import convert_images_with_text

convert_images_with_text('data', num_valid=50, photos_subdir='photos', tfrecords_subdir='tfrecords')

Finished loading word embedding weights.
>> Converting image 215/215 shard 4
>> Converting image 50/50 shard 4

Finished converting the dataset!


In [1]:
from image_model.im_model import fine_tune_model_with_text

dataset_dir = 'data'
checkpoints_dir = 'image_model/pretrained_model'
train_dir = 'image_model/fine_tuned_model'
num_steps = 5
learning_rate = 1e-7
fine_tune_model_with_text(dataset_dir, checkpoints_dir, train_dir,
                          num_steps, learning_rate)

Instructions for updating:
Use tf.losses.softmax_cross_entropy instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 1: loss = 2.5468 (24.51 sec/step)
Step 0: loss: 2.547, validation accuracy: 0.100
INFO:tensorflow:global step 2: loss = 2.3731 (14.32 sec/step)
Step 1: loss: 2.373, validation accuracy: 0.100
INFO:tensorflow:global step 3: loss = 2.4964 (13.87 sec/step)
Step 2: loss: 2.496, validation accuracy: 0.100
INFO:tensorflow:global step 4: loss = 2.4450 (13.65 sec/step)
Step 3: loss: 2.445, validation accuracy: 0.100
INFO:tensorflow:global step 5: loss = 2.5533 (13.69 s

In [34]:
import tensorflow as tf
from tensorflow.contrib import slim
from image_model import inception_v1
from datasets import dataset_utils
from datasets.convert_to_dataset import get_split, get_split_with_text
from image_model.im_model import _load_batch

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset_dir = 'data'
    # Load validation data
    image_size = inception_v1.default_image_size
    dataset_valid = get_split_with_text('validation', dataset_dir)
    images_valid, _, labels_valid = _load_batch(dataset_valid, batch_size=dataset_valid.num_samples, shuffle=False, 
                                                height=image_size, width=image_size)
        
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception_v1.inception_v1_arg_scope()):
        logits_valid, _ = inception_v1.inception_v1(images_valid, num_classes=dataset_valid.num_classes, 
                                                    is_training=False, reuse=True)
    # Accuracy metrics
    accuracy_valid = slim.metrics.streaming_accuracy(tf.cast(labels_valid, tf.int32),
                                           tf.cast(tf.argmax(logits_valid, 1), tf.int32))
    #mse_valid = slim.metrics.accuracy(tf.cast(labels_valid, tf.int32),
     #                                           tf.cast(tf.argmax(logits_valid, 1), tf.int32))

    # Choose the metrics to compute:
    names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
        'http://localhost:8888/notebooks/Documents/tumblr-sentiment/Word%20embedding.ipynb#accuracy': accuracy_valid,
    })
        
    checkpoint_dir = 'image_model/fine_tuned_model'
    log_dir = '/tmp/my_model_eval/'
    # We'll evaluate 1000 batches:
    num_evals = 1
    # Evaluate every 10 minutes:
    slim.evaluation.evaluation_loop(
      '',
      checkpoint_dir,
      log_dir,
      num_evals=num_evals,
      eval_op=names_to_updates.values(),
      eval_interval_secs=600)

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
INFO:tensorflow:Waiting for new checkpoint at image_model/fine_tuned_model
INFO:tensorflow:Found new checkpoint at image_model/fine_tuned_model/model.ckpt-5
INFO:tensorflow:Starting evaluation at 2017-08-30-18:26:06
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-08-30-18:26:14


KeyboardInterrupt: 

In [ ]:
#https://stackoverflow.com/questions/39732460/how-to-use-evaluation-loop-with-train-loop-in-tf-slim